In [ ]:
import os

# Select the backend before importing folps.py
os.environ["FOLPS_BACKEND"] = "numpy"  #'numpy' or 'jax'

from folps_newtest import *
from cosmo_class import *
from use_fkpt import *

In [ ]:
#omega_i = Omega_i h² 
omega_b = 0.022;        #baryons
omega_cdm = 0.122;       #CDM
omega_ncdm = 0.0006442;    #massive neutrinos 
h = 0.6711;                 #h = H0/100
z_pk = 0.3;  #evaluation redshift
z_pk_scale = 0.97
CosmoParams = [z_pk, omega_b, omega_cdm, omega_ncdm, h]

In [ ]:
%%time
#linear power spectrum
ps = generate_ps(h = 0.6711, ombh2 = 0.022, omch2 = 0.122, omnuh2 = 0.0006442, 
            As = 2e-9, ns = 0.965, z = z_pk, z_scale=[0.97],
            N_ur = 2.0328,
            khmin = 0.0001, khmax = 2.0, nbk = 1000, spectra = 'matter')
inputpkT = np.array([ps['kh'], ps['pk']])
kwargs = {'z': z_pk, 'h': h, 'Omega_m': ps['cosmo'].Omega0_m(), 'f0':0.6900011469771554,
          'fnu':ps['cosmo'].scale_independent_growth_factor_f(z_pk)}

In [ ]:
matrix = MatrixCalculator(A_full=True)
mmatrices = matrix.get_mmatrices()

In [ ]:
%%time
#1-loop
nonlinear = NonLinearPowerSpectrumCalculator(mmatrices=mmatrices,
                                             kernels='eds',
                                             **kwargs)

In [ ]:
kwargs

In [ ]:
%%time
table, table_now = nonlinear.calculate_loop_table(k=inputpkT[0], pklin=inputpkT[1], #cosmo=ps['cosmo'],
                                             **kwargs)

In [ ]:
PshotP = 1. / 0.0002118763
# Bias parameters
b1 = 1.645
b2 = -0.46
bs2 = -4./7*(b1 - 1)
b3nl = 32./315*(b1 - 1)
# EFT parameters
alpha0 = 3                 #units: [Mpc/h]^2
alpha2 = -28.9             #units: [Mpc/h]^2
alpha4 = 0.0               #units: [Mpc/h]^2
ctilde = 0.0               #units: [Mpc/h]^4
# Stochatic parameters
alphashot0 = 0.08
alphashot2 = -8.1          #units: [Mpc/h]^2
pars = [b1, b2, bs2, b3nl, alpha0, alpha2, alpha4, ctilde, alphashot0, alphashot2, PshotP]
qpar, qper = 1., 1.


k = np.logspace(np.log10(0.01), np.log10(0.3), num=100) # array of  output k in [h/Mpc]

In [ ]:
%%time
multipoles = RSDMultipolesPowerSpectrumCalculator(model='EFT')

In [ ]:
%%time
P0, P2, P4  = multipoles.get_rsd_pkell(kobs=k, qpar=qpar, qper=qper, pars=pars, table=table, table_now=table_now)

In [ ]:
%%time
P0_noir, P2_noir  = multipoles.get_rsd_pkell(kobs=k, qpar=qpar, qper=qper, pars=pars, table=table, table_now=table_now, ells=(0, 2),IR_resummation=False, bias_scheme="folps")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig, axs = plt.subplots(figsize=(7,5))
axs.set_xlabel(r'$k \, [h\, Mpc^{-1}]$', fontsize =  14)
axs.set_ylabel(r'$k P_{\ell}(k) \, [h^{-1} \,  Mpc]^2$', fontsize =  14)

axs.plot(k, k * P0, color = 'navy', ls='-', label=r'$\ell = 0$')
axs.plot(k, k * P0_noir, color = 'navy', ls='-.')

axs.plot(k, k * P2, color = 'maroon', ls='-', label=r'$\ell = 2$')
axs.plot(k, k * P2_noir, color = 'maroon', ls='-.')
axs.plot(k, k * P4, color = 'darkgreen', ls='-', label=r'$\ell = 4$')

axs.set_xlim([k[0], 0.2])
#axs.set_ylim([-200.0, 1800])

leg1 = axs.legend(loc = 'best');
leg1.get_frame().set_linewidth(0.0);

plt.show()

In [ ]:
multipoles.model

In [ ]:
len(table)

In [ ]:
table_now[32]

In [ ]:
table_now[33]

In [ ]:
table_now[34]

In [ ]:
table_now[35]

In [ ]:
np.array([P0, P2])

In [ ]:
############################# Bispectrum ###############################3

In [ ]:
classy  = run_class(h = 0.6711, ombh2 = 0.022, omch2 = 0.122, omnuh2 = 0.0006442, 
                    As = 2e-9, ns = 0.965, z = 0.3, z_scale=[0.97],
                    N_ur = 2.0328,
                    khmin = 0.0001, khmax = 2.0, nbk = 1000, spectra = 'matter')

In [ ]:
#pars for Bispectrum
z_pk=0.0;


b1v = 1;
c1v = 0; b2v = 0.0; bsv = 0;

Omv=classy['cosmo'].Omega0_m();
hv=classy['cosmo'].h(); 
f=classy['cosmo'].scale_independent_growth_factor_f(z_pk)

qparv = 1; qperpv = 1;
Bshotv=0.0
Pshotv=0.0


bisp_nuis_paramsv=[b1v,b2v,bsv,c1v,Bshotv,Pshotv]
bisp_cosmo_paramsv=[Omv, hv, nonlinear.f0]

In [ ]:
ks = np.logspace(np.log10(0.0001), np.log10(0.4), 30)
k1k2T = np.vstack([ks,ks]).T

In [ ]:
%%time
#precision=[8,10,10]
B000,B202=Bisp_Sugiyama(bisp_cosmo_paramsv, bisp_nuis_paramsv, pk_input=nonlinear.inputpkT, z_pk=z_pk, k1k2pairs=k1k2T, Omfid=-1,qpar=1,qperp=1)#,precision=precision)

In [ ]:
fig, axs = plt.subplots(figsize=(7,5))
axs.set_xlabel(r'$k \, [h\, Mpc^{-1}]$', fontsize =  14)
axs.set_ylabel(r'$k^2 B(k,k) $', fontsize =  14)

#axs.plot(k1k2T.T[0], k1k2T.T[0]*k1k2T.T[0]*B000, ls='-', label='B000')
#axs.plot(k1k2T.T[0],k1k2T.T[0]*k1k2T.T[0]*B202, ls='-', label='B202')


axs.loglog(k1k2T.T[0],B000, ls='-', label='B000')
axs.loglog(k1k2T.T[0],B202, ls='-', label='B202')
axs.loglog(nonlinear.inputpkT[0],nonlinear.inputpkT[1]*nonlinear.inputpkT[1], ls='-', label='P^2')



axs.set_xlim([0.0001,0.15])
axs.set_ylim([1e5, 1e10])

leg1 = axs.legend(loc = 'best');
leg1.get_frame().set_linewidth(0.0);

plt.show()

In [ ]:
inputpkT[1]

# 